In [5]:
from random import randint


def roll(*, sides: int = 6, target: int = 4) -> bool:
    return randint(1, sides) >= target


def roll_dice(*, dice: int = 1, sides: int = 6, target: int = 4) -> int:
    return sum(roll(sides=sides, target=target) for _ in range(dice))

In [42]:
from collections import defaultdict
from dataclasses import dataclass


@dataclass(frozen=True)
class Test:
    dice: int
    target: int


@dataclass
class Result:
    successes: int = 0
    failures: int = 0

    @property
    def total(self) -> int:
        return self.successes + self.failures

    def success_percentage(self) -> float:
        return self.successes / self.total

sides = 10
target = 6


max_d = 9
ds = [i+1 for i in range(max_d)]

max_ob = 4
obs = [i+1 for i in range(max_ob)]

tests = 50_000
test_results: dict[Test, Result] = defaultdict(Result)

for d in ds:
    for ob in obs:
        for _ in range(tests):
            if roll_dice(dice=d, sides=sides, target=target) >= ob:
                test_results[Test(d, ob)].successes += 1
            else:
                test_results[Test(d, ob)].failures += 1

In [43]:
import pandas as pd

data = []
for d in ds:
    data.append([*(f"{test_results[Test(d, target)].success_percentage() * 100:.1f}%" for target in obs)])

print(f"Rolling d{sides}>={target}")
pd.DataFrame(data, columns=[*(f"Ob. {i}" for i in obs)], index=[f"D{i}" for i in ds])

Rolling d10>=6


,Ob. 1,Ob. 2,Ob. 3,Ob. 4
D1,49.6%,0.0%,0.0%,0.0%
D2,74.9%,25.1%,0.0%,0.0%
D3,87.7%,50.1%,12.7%,0.0%
D4,93.5%,68.9%,31.2%,6.2%
D5,96.9%,81.2%,50.2%,18.1%
D6,98.6%,89.2%,65.8%,34.3%
D7,99.2%,93.7%,77.2%,49.5%
D8,99.6%,96.5%,85.5%,63.5%
D9,99.8%,98.0%,90.8%,74.8%
